In [1]:
import zipfile

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/archive.zip'

# Path where you want to extract the contents of the zip file
extract_to_path = '/content/drive/MyDrive/Colab Notebooks/'

# Opening the zip file in read mode
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extracting all the contents into the directory specified
    zip_ref.extractall(extract_to_path)

print("Unzipping completed.")


Unzipping completed.


In [4]:
train_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/train'
val_dir = '/content/drive/MyDrive/Colab Notebooks/ultrasound breast classification/val'


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import vit_b_16  # Import ViT
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define your transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the size required by ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load the pre-trained Vision Transformer (ViT) model
model = vit_b_16(pretrained=True)

# Modify the classifier to fit binary classification
model.heads.head = nn.Linear(model.heads.head.in_features, 2)

# Define loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)  # Adjust step_size and gamma as appropriate

# Fine-tuning loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

num_epochs = 5  # Set number of epochs to 5


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    scheduler.step()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / total_predictions
    print(f'Epoch {epoch+1}/{num_epochs} Train loss: {epoch_loss:.4f} Accuracy: {epoch_accuracy:.4f}')

    # Validation loop
    model.eval()
    val_running_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)

    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_accuracy = val_correct_predictions / val_total_predictions
    print(f'Epoch {epoch+1}/{num_epochs} Validation loss: {val_epoch_loss:.4f} Accuracy: {val_epoch_accuracy:.4f}')


# Save the model checkpoint
torch.save(model.state_dict(), 'vit_transformer_finetuned.pth')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:02<00:00, 116MB/s]


Epoch 1/5 Train loss: 0.2195 Accuracy: 0.9096
Epoch 1/5 Validation loss: 0.3702 Accuracy: 0.8633
Epoch 2/5 Train loss: 0.0669 Accuracy: 0.9764
Epoch 2/5 Validation loss: 0.1748 Accuracy: 0.9378
Epoch 3/5 Train loss: 0.0437 Accuracy: 0.9840
Epoch 3/5 Validation loss: 0.2706 Accuracy: 0.9322
Epoch 4/5 Train loss: 0.0300 Accuracy: 0.9900
Epoch 4/5 Validation loss: 0.3524 Accuracy: 0.9089
Epoch 5/5 Train loss: 0.0399 Accuracy: 0.9846
Epoch 5/5 Validation loss: 0.2024 Accuracy: 0.9433


In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import vit_b_16  # Import ViT
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Define your transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the size required by ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Load the pre-trained Vision Transformer (ViT) model
model = vit_b_16(pretrained=True)

# Modify the classifier to fit binary classification, with dropout for regularization
model.heads.head = nn.Sequential(
    nn.Dropout(p=0.5),  # Add dropout with 50% probability
    nn.Linear(model.heads.head.in_features, 2)
)

# Define loss function, optimizer, and scheduler
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)  # Added weight decay for regularization
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

# Fine-tuning loop with early stopping
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

num_epochs = 5
best_val_loss = float('inf')
patience = 2  # Number of epochs to wait after last time validation loss improved.
patience_counter = 0

for epoch in range(num_epochs):
    # Training loop
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

    scheduler.step()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = correct_predictions / total_predictions
    print(f'Epoch {epoch+1}/{num_epochs} Train loss: {epoch_loss:.4f} Accuracy: {epoch_accuracy:.4f}')

    # Validation loop
    model.eval()
    val_running_loss = 0.0
    val_correct_predictions = 0
    val_total_predictions = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct_predictions += (predicted == labels).sum().item()
            val_total_predictions += labels.size(0)

    val_epoch_loss = val_running_loss / len(val_loader.dataset)
    val_epoch_accuracy = val_correct_predictions / val_total_predictions
    print(f'Epoch {epoch+1}/{num_epochs} Validation loss: {val_epoch_loss:.4f} Accuracy: {val_epoch_accuracy:.4f}')


    # Early stopping check
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        patience_counter = 0
        torch.save(model.state_dict(), 'best_vit_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

# Load the best model
model.load_state_dict(torch.load('best_vit_model.pth'))

# Print final results or perform further actions


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/5 Train loss: 0.2106 Accuracy: 0.9110
Epoch 1/5 Validation loss: 0.2353 Accuracy: 0.9078
Epoch 2/5 Train loss: 0.1206 Accuracy: 0.9547
Epoch 2/5 Validation loss: 0.2129 Accuracy: 0.9278
Epoch 3/5 Train loss: 0.0481 Accuracy: 0.9836
Epoch 3/5 Validation loss: 0.1254 Accuracy: 0.9600
Epoch 4/5 Train loss: 0.0509 Accuracy: 0.9806
Epoch 4/5 Validation loss: 0.1600 Accuracy: 0.9378
Epoch 5/5 Train loss: 0.0418 Accuracy: 0.9850
Epoch 5/5 Validation loss: 0.2029 Accuracy: 0.9467
Early stopping triggered at epoch 5


<All keys matched successfully>